# Imports

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# From our Data Directory
from pathlib import Path
data_dir = Path('data')
outputs_dir = Path('outputs')

# Uploading and Cleaning Data

In [2]:
student_data = pd.read_csv(data_dir/'student_data.csv', sep = ';' )

In [3]:
print(student_data.columns)

Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance\t', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd

#### Clean Student Data

In [4]:
# We are going to do an EDA on age at enrollment, debtor (whether the student is going into debt) and admission grade. 
# We will then examine their relationship to our target variable (whether the student dropped out or graduated).

clean_sd = student_data[['Previous qualification (grade)','Debtor', 'Age at enrollment', 
                         'Admission grade', 'Target', 'Tuition fees up to date', "Mother's qualification", "Father's qualification"]]

# Removing the 794 students that are marked as enrolled
clean_sd = clean_sd[clean_sd["Target"] != "Enrolled"]

## Exploring 'Mother's occupation' and 'Father's qualification' for categorization

In [5]:
mothers_quals = sorted(clean_sd["Mother's qualification"].unique())
fathers_quals = sorted(clean_sd["Father's qualification"].unique())

# No 13, 20, 25, 31, 33
print(mothers_quals)

print(fathers_quals)

len(fathers_quals)
# shared_values = set(mothers_quals) & set(fathers_quals)
# shared_values

[1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 14, 18, 19, 22, 26, 27, 29, 30, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
[1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 18, 19, 20, 22, 25, 26, 27, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]


34

In [6]:
allowed_values = {1, 2, 3, 4, 5, 18, 22, 35, 39, 41, 42}

clean_sd_trial = clean_sd[clean_sd["Father's qualification"].isin(allowed_values) & clean_sd["Mother's qualification"].isin(allowed_values)]

In [7]:
clean_sd_trial

,Previous qualification (grade),Debtor,Age at enrollment,Admission grade,Target,Tuition fees up to date,Mother's qualification,Father's qualification
1,160.0,0,19,142.5,Graduate,0,1,3
8,137.0,0,21,129.3,Graduate,1,1,1
13,110.0,0,21,111.8,Graduate,1,1,1
20,122.0,0,21,120.3,Graduate,1,1,1
31,125.0,0,20,130.0,Graduate,1,42,3
...,...,...,...,...,...,...,...,...
4403,137.0,0,19,124.8,Graduate,1,1,1
4404,133.1,1,28,120.0,Dropout,0,2,1
4417,132.0,0,20,133.8,Graduate,1,1,1
4419,125.0,0,19,122.2,Graduate,1,1,1


In [8]:
# 1 - Secondary Education - 12th Year of Schooling or Eq. 
# 2 - Higher Education - Bachelor's Degree 
# 3 - Higher Education - Degree 
# 4 - Higher Education - Master's 
# 5 - Higher Education - Doctorate 
# 6 - Frequency of Higher Education 
# 9 - 12th Year of Schooling - Not Completed 
# 10 - 11th Year of Schooling - Not Completed 
# 11 - 7th Year (Old) 
# 12 - Other - 11th Year of Schooling 
# 14 - 10th Year of Schooling 
# 18 - General commerce course 
# 19 - Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv. 
# 22 - Technical-professional course 
# 26 - 7th year of schooling 
# 27 - 2nd cycle of the general high school course 
# 29 - 9th Year of Schooling - Not Completed 
# 30 - 8th year of schooling 
# 34 - Unknown 
# 35 - Can't read or write 
# 36 - Can read without having a 4th year of schooling 
# 37 - Basic education 1st cycle (4th/5th year) or equiv. 
# 38 - Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv. 
# 39 - Technological specialization course 
# 40 - Higher education - degree (1st cycle) 
# 41 - Specialized higher studies course 
# 42 - Professional higher technical course 
# 43 - Higher Education - Master (2nd cycle) 
# 44 - Higher Education - Doctorate (3rd cycle)

# -------

# Unknown - 0
# Illiterate - 1
# Some Level of Elementary School Education -  2
# Elemenatary School (Completed) - 3
# Middle School Education (Completed) - 4
# High School (Completed) - 5
# Process of Getting Bachelors - 6
# Completed Bachelors - 7
# Process of Getting Masters - 8
# Completed Masters - 9
# Process of Getting Doctorate - 10
# Completed Doctorate - 11

Categorizing the 34 previous categories into 12 categories to better understand the data.

In [9]:
# Complete data including new codes
data = {
    "Code": [1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 14, 18, 19, 22, 26, 27, 29, 30, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 34,
             13, 20, 25, 31, 33],
    "Original Description": [
        "Secondary Education - 12th Year of Schooling or Eq.",
        "Higher Education - Bachelor's Degree",
        "Higher Education - Degree",
        "Higher Education - Master's",
        "Higher Education - Doctorate",
        "Frequency of Higher Education",
        "12th Year of Schooling - Not Completed",
        "11th Year of Schooling - Not Completed",
        "7th Year (Old)",
        "Other - 11th Year of Schooling",
        "10th Year of Schooling",
        "General commerce course",
        "Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv.",
        "Technical-professional course",
        "7th year of schooling",
        "2nd cycle of the general high school course",
        "9th Year of Schooling - Not Completed",
        "8th year of schooling",
        "Can't read or write",
        "Can read without having a 4th year of schooling",
        "Basic education 1st cycle (4th/5th year) or equiv.",
        "Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv.",
        "Technological specialization course",
        "Higher education - degree (1st cycle)",
        "Specialized higher studies course",
        "Professional higher technical course",
        "Higher Education - Master (2nd cycle)",
        "Higher Education - Doctorate (3rd cycle)",
        "Unknown",
        "2nd year complementary high school course",
        "Complementary High School Course",
        "Complementary High School Course – not concluded",
        "General Course of Administration and Commerce",
        "Supplementary Accounting and Administration"
    ],
    "Category": [
        "High School (Completed)",
        "Completed Bachelors",
        "Completed Bachelors",
        "Completed Masters",
        "Completed Doctorate",
        "Process of Getting Bachelors",
        "Middle School Education (Completed)",
        "Middle School Education (Completed)",
        "Elementary School (Completed)",
        "Middle School Education (Completed)",
        "Middle School Education (Completed)",
        "Professional Course",
        "Middle School Education (Completed)",
        "Professional Course",
        "Elementary School (Completed)",
        "High School (Completed)",
        "Middle School Education (Completed)",
        "Middle School Education (Completed)",
        "Illiterate",
        "Some Level of Elementary School Education",
        "Elementary School (Completed)",
        "Some Level of Elementary School Education",
        "Professional Course",
        "Process of Getting Bachelors",
        "Process of Getting Masters",
        "Professional Course",
        "Completed Masters",
        "Completed Doctorate",
        "Unknown",
        "Process of Getting Bachelors",
        "Process of Getting Bachelors",
        "Process of Getting Bachelors",
        "High School (Completed)",
        "Professional Course"
    ]
}

# Mapping categories to numeric codes
category_to_code = {
    "Unknown": 0,
    "Illiterate": 1,
    "Some Level of Elementary School Education": 2,
    "Elementary School (Completed)": 3,
    "Middle School Education (Completed)": 4,
    "High School (Completed)": 5,
    "Process of Getting Bachelors": 6,
    "Completed Bachelors": 7,
    "Process of Getting Masters": 8,
    "Completed Masters": 9,
    "Process of Getting Doctorate": 10,
    "Completed Doctorate": 11,
    "Professional Course": 2  # same as Some Level of Elementary or adjust if desired
}

# Create DataFrame
education_df = pd.DataFrame(data)

# Add numeric category codes
education_df["Category_Code"] = education_df["Category"].map(category_to_code)

# Display
education_df

,Code,Original Description,Category,Category_Code
0,1,Secondary Education - 12th Year of Schooling o...,High School (Completed),5
1,2,Higher Education - Bachelor's Degree,Completed Bachelors,7
2,3,Higher Education - Degree,Completed Bachelors,7
3,4,Higher Education - Master's,Completed Masters,9
4,5,Higher Education - Doctorate,Completed Doctorate,11
5,6,Frequency of Higher Education,Process of Getting Bachelors,6
6,9,12th Year of Schooling - Not Completed,Middle School Education (Completed),4
7,10,11th Year of Schooling - Not Completed,Middle School Education (Completed),4
8,11,7th Year (Old),Elementary School (Completed),3
9,12,Other - 11th Year of Schooling,Middle School Education (Completed),4


Now mapping the codes and new categories to our old clean_sd dataframe.

In [10]:
# Create mapping dictionaries
code_to_category = dict(zip(education_df["Code"], education_df["Category"]))
code_to_code = dict(zip(education_df["Code"], education_df["Category_Code"]))

# Map Mother's qualification
clean_sd["Mother_edu_category"] = clean_sd["Mother's qualification"].map(code_to_category)
clean_sd["Mother_edu_code"] = clean_sd["Mother's qualification"].map(code_to_code).fillna(0).astype(int)

# Map Father's qualification
clean_sd["Father_edu_category"] = clean_sd["Father's qualification"].map(code_to_category)
clean_sd["Father_edu_code"] = clean_sd["Father's qualification"].map(code_to_code).fillna(0).astype(int)

# New DF
clean_sd

,Previous qualification (grade),Debtor,Age at enrollment,Admission grade,Target,Tuition fees up to date,Mother's qualification,Father's qualification,Mother_edu_category,Mother_edu_code,Father_edu_category,Father_edu_code
0,122.0,0,20,127.3,Dropout,1,19,12,Middle School Education (Completed),4,Middle School Education (Completed),4
1,160.0,0,19,142.5,Graduate,0,1,3,High School (Completed),5,Completed Bachelors,7
2,122.0,0,19,124.8,Dropout,0,37,37,Elementary School (Completed),3,Elementary School (Completed),3
3,122.0,0,20,119.6,Graduate,1,38,37,Some Level of Elementary School Education,2,Elementary School (Completed),3
4,100.0,0,45,141.5,Graduate,1,37,38,Elementary School (Completed),3,Some Level of Elementary School Education,2
...,...,...,...,...,...,...,...,...,...,...,...,...
4419,125.0,0,19,122.2,Graduate,1,1,1,High School (Completed),5,High School (Completed),5
4420,120.0,1,18,119.0,Dropout,0,1,1,High School (Completed),5,High School (Completed),5
4421,154.0,0,30,149.5,Dropout,1,37,37,Elementary School (Completed),3,Elementary School (Completed),3
4422,180.0,0,20,153.8,Graduate,1,37,37,Elementary School (Completed),3,Elementary School (Completed),3


### Exploring Data

In [11]:
# 139 parents have an unknown level of education
# Students "enrolled" ~ 800

# Dataframe without the 'unknown' category
clean_sd_education = clean_sd[
    (clean_sd["Mother's qualification"] != 34) &
    (clean_sd["Father's qualification"] != 34)
]

clean_sd_education

,Previous qualification (grade),Debtor,Age at enrollment,Admission grade,Target,Tuition fees up to date,Mother's qualification,Father's qualification,Mother_edu_category,Mother_edu_code,Father_edu_category,Father_edu_code
0,122.0,0,20,127.3,Dropout,1,19,12,Middle School Education (Completed),4,Middle School Education (Completed),4
1,160.0,0,19,142.5,Graduate,0,1,3,High School (Completed),5,Completed Bachelors,7
2,122.0,0,19,124.8,Dropout,0,37,37,Elementary School (Completed),3,Elementary School (Completed),3
3,122.0,0,20,119.6,Graduate,1,38,37,Some Level of Elementary School Education,2,Elementary School (Completed),3
4,100.0,0,45,141.5,Graduate,1,37,38,Elementary School (Completed),3,Some Level of Elementary School Education,2
...,...,...,...,...,...,...,...,...,...,...,...,...
4419,125.0,0,19,122.2,Graduate,1,1,1,High School (Completed),5,High School (Completed),5
4420,120.0,1,18,119.0,Dropout,0,1,1,High School (Completed),5,High School (Completed),5
4421,154.0,0,30,149.5,Dropout,1,37,37,Elementary School (Completed),3,Elementary School (Completed),3
4422,180.0,0,20,153.8,Graduate,1,37,37,Elementary School (Completed),3,Elementary School (Completed),3


Previous qualification (grade) & Admission grade is from a scale of 0 - 200



Debtor = 1 means that yes they going into debt



Tuition fees up to date = 1 means that yes they are up to date

In [12]:
num_mom_cat = clean_sd['Mother_edu_category'].value_counts()
num_dad_cat = clean_sd['Father_edu_category'].value_counts()

In [13]:
num_mom_cat

Mother_edu_category
High School (Completed)                      866
Elementary School (Completed)                844
Middle School Education (Completed)          835
Some Level of Elementary School Education    474
Completed Bachelors                          397
Unknown                                      127
Completed Masters                             37
Completed Doctorate                           17
Professional Course                           12
Process of Getting Bachelors                  12
Process of Getting Masters                     6
Illiterate                                     3
Name: count, dtype: int64

In [14]:
num_dad_cat

Father_edu_category
Elementary School (Completed)                1022
Middle School Education (Completed)           836
High School (Completed)                       734
Some Level of Elementary School Education     583
Completed Bachelors                           263
Unknown                                       109
Completed Masters                              34
Professional Course                            20
Completed Doctorate                            16
Process of Getting Bachelors                   10
Illiterate                                      2
Process of Getting Masters                      1
Name: count, dtype: int64

In [117]:
# Functions to explore data

def print_target_counts(df, col, label):
    counts = df[col].value_counts()

    print(f"{label} Count")
    print(counts)
    print()

    return counts


def filter_by_parent_edu(df, mother_code, father_code=None):
    if father_code is None:
        father_code = mother_code

    return df[
        (df["Mother_edu_code"] == mother_code) &
        (df["Father_edu_code"] == father_code)
    ]

def filter_one_parent(df, parent, parent_code):
    return df[
        (df[parent] == parent_code)
    ]

def print_percent(count_series, parent_label, parent_edu_label, pos_label, neg_label):
    total = count_series.iloc[0] + count_series.iloc[1]

    grad_percent = (count_series.iloc[0] / total) * 100
    drop_percent = (count_series.iloc[1] / total) * 100

    print(f"Students whose {parent_label} completed {parent_edu_label} and {pos_label}: {grad_percent:.2f}%")
    print(f"Students whose {parent_label} completed {parent_edu_label} and {neg_label}: {drop_percent:.2f}%")
    print()


In [118]:
# Both Parents must have both codes
clean_sd_elemen_sl = filter_by_parent_edu(clean_sd, 2)
clean_sd_elementary = filter_by_parent_edu(clean_sd, 3)
clean_sd_middle = filter_by_parent_edu(clean_sd, 4)
clean_sd_high = filter_by_parent_edu(clean_sd, 5)
clean_sd_bachelor = filter_by_parent_edu(clean_sd, 7)

In [56]:
# Moms must have the code
clean_sd_elemen_sl_m = filter_one_parent(clean_sd, "Mother_edu_code", 2)
clean_sd_elementary_m = filter_one_parent(clean_sd, "Mother_edu_code", 3)
clean_sd_middle_m = filter_one_parent(clean_sd, "Mother_edu_code", 4)
clean_sd_high_m = filter_one_parent(clean_sd, "Mother_edu_code", 5)
clean_sd_bachelor_m = filter_one_parent(clean_sd, "Mother_edu_code", 7)

In [57]:
# Dads must have the code
clean_sd_elemen_sl_f = filter_one_parent(clean_sd, "Father_edu_code", 2)
clean_sd_elementary_f = filter_one_parent(clean_sd, "Father_edu_code", 3)
clean_sd_middle_f = filter_one_parent(clean_sd, "Father_edu_code", 4)
clean_sd_high_f = filter_one_parent(clean_sd, "Father_edu_code", 5)
clean_sd_bachelor_f = filter_one_parent(clean_sd, "Father_edu_code", 7)

In [95]:
# Both Parents Counts
elem_sl_count = print_target_counts(clean_sd_elemen_sl, 'Target', "Elementary School Some Level")
elem_count = print_target_counts(clean_sd_elementary, 'Target', "Elementary School")
midd_count = print_target_counts(clean_sd_middle, 'Target', "Middle School")
high_count = print_target_counts(clean_sd_high, 'Target', "High School")
bach_count = print_target_counts(clean_sd_bachelor,'Target', "Bachelors")

Elementary School Some Level Count
Target
Graduate    130
Dropout      49
Name: count, dtype: int64

Elementary School Count
Target
Graduate    308
Dropout     286
Name: count, dtype: int64

Middle School Count
Target
Graduate    209
Dropout     118
Name: count, dtype: int64

High School Count
Target
Graduate    212
Dropout     132
Name: count, dtype: int64

Bachelors Count
Target
Graduate    76
Dropout     57
Name: count, dtype: int64



In [105]:
# Percent Graduated for All Parents
print_percent(elem_sl_count, "parents", "Some Level of Elementary School", "Graduated", "Dropped Out")
print_percent(elem_count,"parents", "Elementary School", "Graduated", "Dropped Out")
print_percent(midd_count,"parents", "Middle School", "Graduated", "Dropped Out")
print_percent(high_count, "parents","High School", "Graduated", "Dropped Out")
print_percent(bach_count, "parents", "their Bachelors", "Graduated", "Dropped Out")

Students whose parents completed Some Level of Elementary School and Graduated: 72.63%
Students whose parents completed Some Level of Elementary School and Dropped Out: 27.37%

Students whose parents completed Elementary School and Graduated: 51.85%
Students whose parents completed Elementary School and Dropped Out: 48.15%

Students whose parents completed Middle School and Graduated: 63.91%
Students whose parents completed Middle School and Dropped Out: 36.09%

Students whose parents completed High School and Graduated: 61.63%
Students whose parents completed High School and Dropped Out: 38.37%

Students whose parents completed their Bachelors and Graduated: 57.14%
Students whose parents completed their Bachelors and Dropped Out: 42.86%



In [97]:
# Counts for Mother
elem_sl_m_count = print_target_counts(clean_sd_elemen_sl_m, 'Target', "Elementary School Some Level (Mother)")
elem_m_count = print_target_counts(clean_sd_elementary_m, 'Target', "Elementary School (Mother)")
midd_m_count = print_target_counts(clean_sd_middle_m, 'Target', "Middle School (Mother)")
high_m_count = print_target_counts(clean_sd_high_m, 'Target', "High School (Mother)")
bach_m_count = print_target_counts(clean_sd_bachelor_m,'Target',  "Bachelors (Mother)")

Elementary School Some Level (Mother) Count
Target
Graduate    337
Dropout     149
Name: count, dtype: int64

Elementary School (Mother) Count
Target
Graduate    459
Dropout     385
Name: count, dtype: int64

Middle School (Mother) Count
Target
Graduate    532
Dropout     303
Name: count, dtype: int64

High School (Mother) Count
Target
Graduate    565
Dropout     301
Name: count, dtype: int64

Bachelors (Mother) Count
Target
Graduate    238
Dropout     159
Name: count, dtype: int64



In [106]:
# Percent Graduated for Moms
print_percent(elem_sl_m_count, "Mothers", "Some Level of Elementary School", "Graduated", "Dropped Out")
print_percent(elem_m_count, "Mothers", "Elementary School", "Graduated", "Dropped Out")
print_percent(midd_m_count,"Mothers", "Middle School", "Graduated", "Dropped Out")
print_percent(high_m_count, "Mothers","High School", "Graduated", "Dropped Out")
print_percent(bach_m_count, "Mothers","their Bachelors", "Graduated", "Dropped Out")

Students whose Mothers completed Some Level of Elementary School and Graduated: 69.34%
Students whose Mothers completed Some Level of Elementary School and Dropped Out: 30.66%

Students whose Mothers completed Elementary School and Graduated: 54.38%
Students whose Mothers completed Elementary School and Dropped Out: 45.62%

Students whose Mothers completed Middle School and Graduated: 63.71%
Students whose Mothers completed Middle School and Dropped Out: 36.29%

Students whose Mothers completed High School and Graduated: 65.24%
Students whose Mothers completed High School and Dropped Out: 34.76%

Students whose Mothers completed their Bachelors and Graduated: 59.95%
Students whose Mothers completed their Bachelors and Dropped Out: 40.05%



In [110]:
# Counts for Fathers
elem_sl_f_count = print_target_counts(clean_sd_elemen_sl_f, 'Target', "Elementary School Some Level (Father)")
elem_f_count = print_target_counts(clean_sd_elementary_f, 'Target',"Elementary School (Father)")
midd_f_count = print_target_counts(clean_sd_middle_f, 'Target',"Middle School (Father)")
high_f_count = print_target_counts(clean_sd_high_f, 'Target',"High School (Father)")
bach_f_count = print_target_counts(clean_sd_bachelor_f, 'Target', "Bachelors (Father)")

Elementary School Some Level (Father) Count
Target
Graduate    417
Dropout     186
Name: count, dtype: int64

Elementary School (Father) Count
Target
Graduate    585
Dropout     437
Name: count, dtype: int64

Middle School (Father) Count
Target
Graduate    550
Dropout     286
Name: count, dtype: int64

High School (Father) Count
Target
Graduate    451
Dropout     283
Name: count, dtype: int64

Bachelors (Father) Count
Target
Graduate    151
Dropout     112
Name: count, dtype: int64



In [111]:
# Percent Graduated for Fathers
print_percent(elem_sl_f_count, "Fathers", "Some Level of Elementary School", "Graduated", "Dropped Out")
print_percent(elem_f_count, "Fathers", "Elementary School", "Graduated", "Dropped Out")
print_percent(midd_f_count,"Fathers", "Middle School", "Graduated", "Dropped Out")
print_percent(high_f_count, "Fathers","High School", "Graduated", "Dropped Out")
print_percent(bach_f_count, "Fathers","their Bachelors", "Graduated", "Dropped Out")

Students whose Fathers completed Some Level of Elementary School and Graduated: 69.15%
Students whose Fathers completed Some Level of Elementary School and Dropped Out: 30.85%

Students whose Fathers completed Elementary School and Graduated: 57.24%
Students whose Fathers completed Elementary School and Dropped Out: 42.76%

Students whose Fathers completed Middle School and Graduated: 65.79%
Students whose Fathers completed Middle School and Dropped Out: 34.21%

Students whose Fathers completed High School and Graduated: 61.44%
Students whose Fathers completed High School and Dropped Out: 38.56%

Students whose Fathers completed their Bachelors and Graduated: 57.41%
Students whose Fathers completed their Bachelors and Dropped Out: 42.59%



In [30]:
# Parents who graduated & students who have debt 

In [112]:
# Counts for Each Level of Education
elem_sl_tuition = print_target_counts(clean_sd_elemen_sl, 'Tuition fees up to date', "Elementary School Some Level")
elem_tuition = print_target_counts(clean_sd_elementary, 'Tuition fees up to date', "Elementary School")
midd_tuition = print_target_counts(clean_sd_middle, 'Tuition fees up to date', "Middle School")
high_tuition = print_target_counts(clean_sd_high, 'Tuition fees up to date', "High School")
bach_tuition = print_target_counts(clean_sd_bachelor,'Tuition fees up to date', "Bachelors")

Elementary School Some Level Count
Tuition fees up to date
1    167
0     12
Name: count, dtype: int64

Elementary School Count
Tuition fees up to date
1    483
0    111
Name: count, dtype: int64

Middle School Count
Tuition fees up to date
1    290
0     37
Name: count, dtype: int64

High School Count
Tuition fees up to date
1    294
0     50
Name: count, dtype: int64

Bachelors Count
Tuition fees up to date
1    116
0     17
Name: count, dtype: int64



In [113]:
# Percent w Tuition Fees up to date for All Parents
print_percent(elem_sl_tuition, "parents", "Some Level of Elementary School","have Tuition fees up to date", "don't have Tuition fees up to date")
print_percent(elem_tuition,"parents", "Elementary School","have Tuition fees up to date", "don't have Tuition fees up to date")
print_percent(midd_tuition,"parents", "Middle School","have Tuition fees up to date", "don't have Tuition fees up to date")
print_percent(high_tuition, "parents","High School","have Tuition fees up to date", "don't have Tuition fees up to date")
print_percent(bach_tuition, "parents", "their Bachelors","have Tuition fees up to date", "don't have Tuition fees up to date")

Students whose parents completed Some Level of Elementary School and have Tuition fees up to date: 93.30%
Students whose parents completed Some Level of Elementary School and don't have Tuition fees up to date: 6.70%

Students whose parents completed Elementary School and have Tuition fees up to date: 81.31%
Students whose parents completed Elementary School and don't have Tuition fees up to date: 18.69%

Students whose parents completed Middle School and have Tuition fees up to date: 88.69%
Students whose parents completed Middle School and don't have Tuition fees up to date: 11.31%

Students whose parents completed High School and have Tuition fees up to date: 85.47%
Students whose parents completed High School and don't have Tuition fees up to date: 14.53%

Students whose parents completed their Bachelors and have Tuition fees up to date: 87.22%
Students whose parents completed their Bachelors and don't have Tuition fees up to date: 12.78%



In [116]:
# Debt Counts
elem_sl_debt = print_target_counts(clean_sd_elemen_sl, 'Debtor', "Elementary School Some Level")
elem_debt = print_target_counts(clean_sd_elementary, 'Debtor', "Elementary School")
midd_debt = print_target_counts(clean_sd_middle, 'Debtor', "Middle School")
high_debt = print_target_counts(clean_sd_high, 'Debtor', "High School")
bach_debt = print_target_counts(clean_sd_bachelor,'Debtor', "Bachelors")

Elementary School Some Level Count
Debtor
0    160
1     19
Name: count, dtype: int64

Elementary School Count
Debtor
0    518
1     76
Name: count, dtype: int64

Middle School Count
Debtor
0    292
1     35
Name: count, dtype: int64

High School Count
Debtor
0    297
1     47
Name: count, dtype: int64

Bachelors Count
Debtor
0    119
1     14
Name: count, dtype: int64



In [115]:
# Percent w no debt for All Parents
print_percent(elem_sl_debt, "parents", "Some Level of Elementary School","have no Debt", "have Debt")
print_percent(elem_debt,"parents", "Elementary School","have no Debt", "have Debt")
print_percent(midd_debt,"parents", "Middle School","have no Debt", "have Debt")
print_percent(high_debt, "parents","High School","have no Debt", "have Debt")
print_percent(bach_debt, "parents", "their Bachelors","have no Debt", "have Debt")

Students whose parents completed Some Level of Elementary School and have no Debt: 89.39%
Students whose parents completed Some Level of Elementary School and have Debt: 10.61%

Students whose parents completed Elementary School and have no Debt: 87.21%
Students whose parents completed Elementary School and have Debt: 12.79%

Students whose parents completed Middle School and have no Debt: 89.30%
Students whose parents completed Middle School and have Debt: 10.70%

Students whose parents completed High School and have no Debt: 86.34%
Students whose parents completed High School and have Debt: 13.66%

Students whose parents completed their Bachelors and have no Debt: 89.47%
Students whose parents completed their Bachelors and have Debt: 10.53%

